In [7]:
!wget https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip

--2025-09-23 07:38:04--  https://opencorpora.org/files/export/dict/dict.opcorpora.xml.zip
Resolving opencorpora.org (opencorpora.org)... 172.67.163.210, 104.21.15.199, 2606:4700:3030::6815:fc7, ...
Connecting to opencorpora.org (opencorpora.org)|172.67.163.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28757314 (27M) [application/zip]
Saving to: ‘dict.opcorpora.xml.zip’

dict.opcorpora.xml. 100%[===================>]  27.42M  --.-KB/s    in 0.1s    

2025-09-23 07:38:05 (191 MB/s) - ‘dict.opcorpora.xml.zip’ saved [28757314/28757314]



In [8]:
!unzip dict.opcorpora.xml.zip

Archive:  dict.opcorpora.xml.zip
  inflating: dict.opcorpora.xml      


In [9]:
import xml.etree.ElementTree as ET
import re

In [10]:
class TextPreprocess:
    def __init__(self, path_to_xml):
        opcorpa_tree = ET.parse(path_to_xml)
        opcorpa_tree_root = opcorpa_tree.getroot()

        self.gramm_dict = {
            "NOUN": "S",   # существительное
            "NPRO": "NI",  # местоимение-существительное

            "ADJF": "A",   # прилагательное (полное)
            "ADJS": "A",   # прилагательное (краткое)
            "COMP": "A",   # компаратив
            "ADJ": "A",

            "VERB": "V",   # глагол
            "INFN": "V",   # инфинитив
            "GRND": "V",   # деепричастие - глагольная форма
            "PRTF": "V",   # причастие (полное)
            "PRTS": "V",   # причастие (краткое)

            "ADVB": "ADV", # наречие
            "ADV": "ADV", # наречие

            "NUMR": "NUM", # числительное
            "PREP": "PR",  # предлог
            "CONJ": "CONJ",# союз
            "PRCL": "PART",# частица
            "INTJ": "INTJ" # междометие
        }
        self.pos_gramm = self.gramm_dict.keys()

        self.wordform_to_lemma = {}
        self.create_detailed_dictionary_(opcorpa_tree_root)

    @staticmethod
    def normalize_word_(word):
        return word.strip().lower().replace('ё', 'е')

    def create_detailed_dictionary_(self, xml_root):
        self.wordform_to_lemma = {}
        for lemma in xml_root.findall('.//lemma'):
            # пытаемся получить текст леммы
            lemma_text = lemma.get('t')
            # если не вышло, то берем первую <l ...>
            if lemma_text is None:
                head = lemma.find('l')
                if head is not None:
                    lemma_text = head.get('t')

                    # собираем все грамм. теги формы
                    tags = []
                    for g in head.findall('g'):
                        v = g.get('v')
                        if v:
                            tags.append(v.upper())

                    # получаем POS
                    lemma_pos = "OTHER"
                    for t in tags:
                        if t in self.pos_gramm:
                            lemma_pos = self.gramm_dict.get(t, "OTHER")
                            break
                else:
                    lemma_text = None
            if not lemma_text:
                continue

            # собираем все <l> (типовая) и все <f> (флексии)
            form_nodes = []
            form_nodes.extend(lemma.findall('l'))
            form_nodes.extend(lemma.findall('f'))

            # формируем результат
            seen = set()
            for form in form_nodes:
                wf = form.get('t')
                if not wf:
                    continue
                key = self.normalize_word_(wf)
                if (key, lemma_text) in seen:
                    continue
                seen.add((key, lemma_text))

                entry = {'lemma': lemma_text, 'pos': lemma_pos}
                if key in self.wordform_to_lemma.keys():
                    if entry not in self.wordform_to_lemma[key]:
                        self.wordform_to_lemma[key].append(entry)
                else:
                    self.wordform_to_lemma[key] = [entry]

    def pipeline(self, text):
        # удаляем знаки
        text_without_punctuation = self._delete_punctuation_marks(text)
        # разбиваем на слова
        words = self._text_to_words(text_without_punctuation)
        # получае леммы для слов
        results = self._get_words_lemma(words)
        return results

    @staticmethod
    def _delete_punctuation_marks(text):
        return re.sub(r'[^\w\s]', '', text)

    @staticmethod
    def _text_to_words(text):
        return [w for w in text.split() if w]

    def _get_words_lemma(self, words):
        results = []
        for word in words:
            word_lower = word.lower()
            lemmas = self.wordform_to_lemma.get(self.normalize_word_(word))
            result = {
                "word": word,
                "lemmas": lemmas
            }
            results.append(result)
        return results


In [12]:
tp = TextPreprocess("dict.opcorpora.xml")

In [13]:
# ВВЕДИТЕ ЗДЕСЬ ТЕКСТ
text = "Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников. Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры."

results = tp.pipeline(text)
for result in results:
    word = result["word"]
    lemmas = result["lemmas"]
    result_string = word + "("
    for idx, lemma in enumerate(lemmas):
        result_string += lemma["lemma"] + "=" + lemma["pos"]
        result_string += "," if idx != len(lemmas)-1 else ""
    result_string += ")"
    print(result_string)

Стала(стал=V)
стабильнее(стабильнее=A)
экономическая(экономический=A)
и(и=CONJ,и=INTJ,и=PART,и=S)
политическая(политический=A)
обстановка(обстановка=S)
предприятия(предприятие=S)
вывели(вывел=V)
из(из=PR,иза=S)
тени(теню=V,тень=S)
зарплаты(зарплата=S)
сотрудников(сотрудник=S)
Все(весь=A,всё=PART)
Гришины(гришины=S,гришин=A)
одноклассники(одноклассник=S)
уже(уж=S,уже=ADV,уже=A,уже=PART)
побывали(побывал=V)
за(за=PR)
границей(граница=S)
он(он=NI)
был(есть=V)
чуть(чуть=ADV,чуть=CONJ)
ли(ли=CONJ,ли=PART,ли=S)
не(не=PART)
единственным(единственный=A)
кого(кто=NI)
не(не=PART)
вывозили(вывожу=V,вывозил=V)
никуда(никуда=ADV)
дальше(дальше=A)
Красной(красная=S,красный=A)
Пахры(пахра=S)
